# Actividad Integradora

## Descripción del problema

¡Felicidades! Eres el orgulloso propietario de 5 robots nuevos y un almacén lleno de cajas. El dueño anterior del almacén lo dejó en completo desorden, por lo que depende de tus robots organizar las cajas en algo parecido al orden y convertirlo en un negocio exitoso. 

Cada  robot  está  equipado  con  ruedas  omnidireccionales  y,  por  lo  tanto,  puede  conducir  en  las cuatro   direcciones.   Pueden   recoger   cajas   en   celdas   de   cuadrícula   adyacentes   con   sus manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es una pared, contiene una pila de cajas(y cuantas cajas hay en la pila)o está ocupado por otro robot. Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese momento. 

Lamentablemente, tu  presupuesto  resultó  insuficiente  para  adquirirun  software  de  gestión  de agentes múltiples de última generación. Pero eso no debería ser un gran problema ... ¿verdad? Tu tarea es enseñar a sus robots cómo ordenar su almacén. La organización delosagentesdepende de ti, siempre que todas las cajas terminen en pilas ordenadas de cinco.

Realiza la siguiente simulación:
* Inicializa las posiciones iniciales de las Kcajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas.
* Todos los agentes empiezan en posición aleatorias vacías.
* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:

* Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.
* Número de movimientos realizados por todos los robots.
* Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.

In [1]:
!pip3 install mesa
!pip3 install numpy
!pip3 install matplotlib

In [2]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [3]:
def obtener_almacen(modelo):
    almacen = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Robot):
                almacen[x][y] = 6
            elif isinstance(contenido, Caja):
                almacen[x][y] = contenido.stack_size
    return almacen
    
class Robot(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.nueva_posicion = None
        self.caja = None
        self.caja_coordenadas = None
        self.movimientos = 0
        
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)
        # Defino el siguiente estado que va a tener el piso para la siguiente iteracion sin asignarlo todavia eso lo hago en el método `advance`.
        contador = 0
        self.nueva_posicion = self.pos
        if self.caja == None:
            for vecino in vecinos:
                if isinstance(vecino, Caja):
                    contador += 1
                    vecino.siguiente_stack_size = vecino.stack_size
                    if vecino.siguiente_stack_size > 0 and vecino.siguiente_stack_size < 5 and self.caja_coordenadas != None and self.caja_coordenadas != vecino.pos:
                        self.caja = 1
                        vecino.siguiente_stack_size -= 1
                        self.nueva_posicion = self.pos
                        break
                    elif self.caja_coordenadas == None and vecino.siguiente_stack_size < 5 and vecino.siguiente_stack_size != 0:
                        self.caja_coordenadas = vecino.pos
                        self.nueva_posicion = self.pos
                        break
                    elif contador >= 2:
                        vecindario = self.model.grid.get_neighbors(
                            self.pos,
                            moore=False,
                            include_center=False)
                        temp_list = []
                        for vecino2 in vecindario:
                            if len(self.model.grid.get_cell_list_contents(vecino2.pos)) == 1 and vecino2.stack_size == 0:
                                #if not self.agentClose(vecino2):
                                    print("Celda cercana")
                                    print(vecino2)
                                    print(vecino2.pos)
                                    temp_list.append(vecino2.pos)
                        if len(temp_list) != 0:
                            nueva_posicion = self.random.choice(temp_list)
                            self.nueva_posicion = nueva_posicion
                            print(self.nueva_posicion)
        else:
            x2, y2 = self.caja_coordenadas
            x1, y1 = self.pos
            x = (x2 - x1)
            y = (y2 - y1)
            if abs(x) + abs(y) == 1:
                temp_cell_list = self.model.grid.get_cell_list_contents(self.caja_coordenadas)
                vecino5 = temp_cell_list[0]
                if vecino5.stack_size < 5:
                    vecino5.siguiente_stack_size = vecino5.stack_size + 1
                    self.caja = None
                    if vecino5.siguiente_stack_size == 5:
                        self.caja_coordenadas = None
                else:
                    vecinos = self.model.grid.get_neighbors(
                        self.pos,
                        moore=False,
                        include_center=False)
                    for vecino3 in vecinos:
                        temp_cell_list2 = self.model.grid.get_cell_list_contents(vecino3.pos)
                        if len(temp_cell_list2) == 1 and vecino3.stack_size == 0:
                            #if not self.agentClose(vecino3):
                                vecino3.siguiente_stack_size = 1
                                self.caja = None
                                self.caja_coordenadas = vecino3.pos
                                break
                    
            else:
                print("Going to coordinates")
                temp_pos_x = 0
                temp_pos_y = 0
                if abs(x) > 0:
                    x = int(x / np.linalg.norm(x))
                    temp_pos_x = (x1 + x, y1)
                if abs(y) > 0:
                    y = int(y / np.linalg.norm(y))
                    temp_pos_y = (x1, y1 + y)
                temp_pos = [temp_pos_x, temp_pos_y]
                self.nueva_posicion = self.pos
                if temp_pos_x != 0 or temp_pos_y != 0:
                    i = self.random.choice(temp_pos)
                    if i != 0:
                        temp_cell_list = self.model.grid.get_cell_list_contents(i)
                        if len(temp_cell_list) == 1 and temp_cell_list[0].stack_size == 0:  
                            #if not self.agentClose(temp_cell_list[0]):
                                self.nueva_posicion = i
                            #else:
                            #    vecindario = self.model.grid.get_neighbors(
                            #        self.pos,
                            #        moore=False,
                            #       include_center=False)
                            #    temp_list = []
                            #    for vecino9 in vecindario:
                            #        if len(self.model.grid.get_cell_list_contents(vecino9.pos)) == 1 and vecino9.stack_size == 0:
                            #            #if not self.agentClose(vecino9):
                            #                temp_list.append(vecino9.pos)
                            #    if len(temp_list) != 0:
                            #        nueva_posicion = self.random.choice(temp_list)
                            #        self.nueva_posicion = nueva_posicion
                        else:
                            vecindario = self.model.grid.get_neighbors(
                                self.pos,
                                moore=False,
                                include_center=False)
                            temp_list = []
                            for vecino8 in vecindario:
                                if len(self.model.grid.get_cell_list_contents(vecino8.pos)) == 1 and vecino8.stack_size == 0:
                                    #if not self.agentClose(vecino8):
                                        temp_list.append(vecino8.pos)
                            if len(temp_list) != 0:
                                nueva_posicion = self.random.choice(temp_list)
                                self.nueva_posicion = nueva_posicion
                else:
                    vecinos = self.model.grid.get_neighbors(
                        self.pos,
                        moore=False,
                        include_center=False)
                    for vecino3 in vecinos:
                        temp_cell_list2 = self.model.grid.get_cell_list_contents(vecino3.pos)
                        if len(temp_cell_list2) == 1 and vecino3.stack_size == 0:
                            #if not self.agentClose(vecino3):
                                vecino3.siguiente_stack_size = 1
                                self.caja = None
                                self.caja_coordenadas = vecino3.pos
                                break
        self.advance()
        
    def advance(self):
        # Actualizamos el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)
        
        for vecino in vecinos:
            if isinstance(vecino, Caja) and vecino.siguiente_stack_size != None: 
                vecino.stack_size = vecino.siguiente_stack_size
                vecino.siguiente_stack_size = None
        
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            
        # Movemos la aspiradora a su nueva posicion
        print("Nueva posicion final")
        print(self.nueva_posicion)
        self.model.grid.move_agent(self, self.nueva_posicion)
        
    #def agentClose(self, celda):
    #    vecinos = self.model.grid.get_neighbors(
    #        celda.pos,
    #        moore=False,
    #        include_center=False)
    #    for vecino3 in vecinos:
    #        if isinstance(vecino3, Robot) and vecino3.pos != self.pos:
    #            print("Agente adyacente")
    #            print(vecino3.pos)
    #            return True
    #    return False
    
class Caja(Agent):
    # Cantidad máxima de cajas 5
    
    def __init__(self, pos, modelo, stack_size=0):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.stack_size = stack_size
        self.siguiente_stack_size = None

class Almacen(Model):
    
    def __init__(self, m, n, num_agentes, num_cajas):
        self.num_agentes = num_agentes
        self.num_cajas = num_cajas
        self.grid = MultiGrid(m, n, False)
        self.schedule = RandomActivation(self)
               
        # Posicionar cajas de forma aleatoria
        lista_celdas_sin_cajas = list(self.grid.empties)
        for celdas in range(num_cajas):
            celda_vacia = random.choice(lista_celdas_sin_cajas)
            caja = Caja(celda_vacia, self)
            caja.stack_size = 1
            self.grid.place_agent(caja, celda_vacia)
            self.schedule.add(caja)
            lista_celdas_sin_cajas.remove(celda_vacia)
        
        # Posicionar celdas sin cajas
        lista_celdas_sin_cajas = list(self.grid.empties)
        for celdas in lista_celdas_sin_cajas:
            caja = Caja(celdas, self)
            self.grid.place_agent(caja, celdas)
            self.schedule.add(caja)
        
        # Posicionar agentes robots
        for i in range(num_agentes):
            celda_vacia = random.choice(lista_celdas_sin_cajas)
            robot = Robot(i, self)
            self.grid.place_agent(robot, celda_vacia)
            self.schedule.add(robot)
            lista_celdas_sin_cajas.remove(celda_vacia)
            
        self.colectordatos = DataCollector(
            model_reporters={'Almacen': obtener_almacen},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def todasceldasordenadas(self):
        celdas_ordenadas = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Caja) and contenido.stack_size == 5:
                    celdas_ordenadas += 1
        if celdas_ordenadas == (self.num_cajas / 5):
            return True
        else:
            return False

In [4]:
# Datos de la habitacion:
M = 8
N = 8

# Numero de agentes
NUM_ROBOTS = 5
NUM_CAJAS = 20

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 4

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Almacen(M, N, NUM_ROBOTS, NUM_CAJAS)
counter = 0

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldasordenadas()):
    print("Counter: ")
    print (counter)
    modelo.step()
    counter += 1
    #print(str(datetime.timedelta(seconds=(time.time() - start_time))))

# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

Counter: 
0
Celda cercana
(5, 6)
Celda cercana
(6, 5)
Celda cercana
(6, 7)
Celda cercana
(7, 6)
(6, 5)
Celda cercana
(5, 6)
Celda cercana
(6, 5)
Celda cercana
(6, 7)
Celda cercana
(7, 6)
(7, 6)
Celda cercana
(5, 6)
Celda cercana
(6, 5)
Celda cercana
(6, 7)
Celda cercana
(7, 6)
(6, 5)
Nueva posicion final
(6, 5)
Celda cercana
(3, 0)
Celda cercana
(4, 1)
Celda cercana
(5, 0)
(3, 0)
Celda cercana
(3, 0)
Celda cercana
(4, 1)
Celda cercana
(5, 0)
(5, 0)
Nueva posicion final
(5, 0)
Nueva posicion final
(0, 3)
Nueva posicion final
(5, 4)
Nueva posicion final
(6, 1)
Counter: 
1
Celda cercana
(0, 2)
(0, 2)
Nueva posicion final
(0, 3)
Nueva posicion final
(6, 5)
Nueva posicion final
(5, 0)
Nueva posicion final
(5, 4)
Celda cercana
(6, 0)
(6, 0)
Nueva posicion final
(6, 1)
Counter: 
2
Nueva posicion final
(6, 1)
Nueva posicion final
(5, 4)
Nueva posicion final
(0, 3)
Celda cercana
(4, 0)
Celda cercana
(6, 0)
(4, 0)
Celda cercana
(4, 0)
Celda cercana
(6, 0)
(6, 0)
Nueva posicion final
(6, 0)
Celda

Celda cercana
(6, 4)
Celda cercana
(7, 3)
(6, 4)
Nueva posicion final
(6, 4)
Celda cercana
(6, 1)
Celda cercana
(7, 0)
Celda cercana
(7, 2)
(7, 2)
Celda cercana
(6, 1)
Celda cercana
(7, 0)
Celda cercana
(7, 2)
(6, 1)
Nueva posicion final
(6, 1)
Celda cercana
(6, 6)
Celda cercana
(7, 5)
(6, 6)
Celda cercana
(6, 6)
Celda cercana
(7, 5)
(7, 5)
Celda cercana
(6, 6)
Celda cercana
(7, 5)
(6, 6)
Nueva posicion final
(6, 6)
Counter: 
25
Celda cercana
(0, 2)
Celda cercana
(1, 3)
Celda cercana
(2, 2)
(1, 3)
Celda cercana
(0, 2)
Celda cercana
(1, 3)
Celda cercana
(2, 2)
(2, 2)
Celda cercana
(0, 2)
Celda cercana
(1, 3)
Celda cercana
(2, 2)
(1, 3)
Nueva posicion final
(1, 3)
Celda cercana
(6, 0)
Celda cercana
(6, 2)
Celda cercana
(7, 1)
(7, 1)
Celda cercana
(6, 0)
Celda cercana
(6, 2)
Celda cercana
(7, 1)
(6, 0)
Celda cercana
(6, 0)
Celda cercana
(6, 2)
Celda cercana
(7, 1)
(6, 0)
Nueva posicion final
(6, 0)
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(6, 5)
Celda cercana
(7, 4)
(5, 4)


Celda cercana
(6, 7)
Celda cercana
(7, 6)
(7, 6)
Celda cercana
(5, 6)
Celda cercana
(6, 5)
Celda cercana
(6, 7)
Celda cercana
(7, 6)
(6, 5)
Nueva posicion final
(6, 5)
Celda cercana
(2, 6)
Celda cercana
(3, 5)
Celda cercana
(4, 6)
(4, 6)
Celda cercana
(2, 6)
Celda cercana
(3, 5)
Celda cercana
(4, 6)
(2, 6)
Celda cercana
(2, 6)
Celda cercana
(3, 5)
Celda cercana
(4, 6)
(2, 6)
Nueva posicion final
(2, 6)
Celda cercana
(3, 3)
Celda cercana
(4, 2)
Celda cercana
(5, 3)
(5, 3)
Celda cercana
(3, 3)
Celda cercana
(4, 2)
Celda cercana
(5, 3)
(4, 2)
Celda cercana
(3, 3)
Celda cercana
(4, 2)
Celda cercana
(5, 3)
(3, 3)
Nueva posicion final
(3, 3)
Counter: 
40
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(7, 4)
(5, 4)
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(7, 4)
(7, 4)
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(7, 4)
(6, 3)
Nueva posicion final
(6, 3)
Celda cercana
(1, 6)
Celda cercana
(2, 5)
Celda cercana
(2, 7)
Celda cercana
(3, 6)
(2, 7)
Celda cercana


Celda cercana
(4, 0)
Celda cercana
(4, 2)
(3, 1)
Nueva posicion final
(3, 1)
Celda cercana
(6, 3)
Celda cercana
(7, 2)
Celda cercana
(7, 4)
(7, 2)
Celda cercana
(6, 3)
Celda cercana
(7, 2)
Celda cercana
(7, 4)
(7, 2)
Nueva posicion final
(7, 2)
Celda cercana
(5, 3)
Celda cercana
(5, 5)
Celda cercana
(6, 4)
(5, 3)
Celda cercana
(5, 3)
Celda cercana
(5, 5)
Celda cercana
(6, 4)
(6, 4)
Celda cercana
(5, 3)
Celda cercana
(5, 5)
Celda cercana
(6, 4)
(6, 4)
Nueva posicion final
(6, 4)
Celda cercana
(6, 7)
Celda cercana
(7, 6)
(7, 6)
Nueva posicion final
(7, 6)
Celda cercana
(1, 3)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
(1, 3)
Celda cercana
(1, 3)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
(1, 3)
Celda cercana
(1, 3)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
(1, 5)
Nueva posicion final
(1, 5)
Counter: 
56
Celda cercana
(6, 2)
Celda cercana
(7, 1)
Celda cercana
(7, 3)
(7, 1)
Celda cercana
(6, 2)
Celda cercana
(7, 1)
Celda cercana
(7, 3)
(6, 2)
Nueva posicion final
(6, 2)
Celda cercana
(5, 4)


(5, 7)
Celda cercana
(7, 7)
(7, 7)
Celda cercana
(5, 7)
Celda cercana
(7, 7)
(7, 7)
Nueva posicion final
(7, 7)
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(6, 5)
Celda cercana
(7, 4)
(6, 3)
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(6, 5)
Celda cercana
(7, 4)
(7, 4)
Celda cercana
(5, 4)
Celda cercana
(6, 3)
Celda cercana
(6, 5)
Celda cercana
(7, 4)
(6, 3)
Nueva posicion final
(6, 3)
Counter: 
69
Celda cercana
(5, 2)
Celda cercana
(6, 1)
Celda cercana
(7, 2)
(7, 2)
Celda cercana
(5, 2)
Celda cercana
(6, 1)
Celda cercana
(7, 2)
(7, 2)
Celda cercana
(5, 2)
Celda cercana
(6, 1)
Celda cercana
(7, 2)
(5, 2)
Nueva posicion final
(5, 2)
Celda cercana
(6, 7)
Celda cercana
(7, 6)
(7, 6)
Nueva posicion final
(7, 6)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
Celda cercana
(6, 4)
Celda cercana
(7, 3)
(6, 4)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
Celda cercana
(6, 4)
Celda cercana
(7, 3)
(6, 2)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
Celda cercana
(6, 4)
Celda cercana


...

Celda cercana
(4, 5)
(3, 4)
Nueva posicion final
(3, 4)
Celda cercana
(0, 0)
Celda cercana
(1, 1)
Celda cercana
(2, 0)
(1, 1)
Celda cercana
(0, 0)
Celda cercana
(1, 1)
Celda cercana
(2, 0)
(0, 0)
Nueva posicion final
(0, 0)
Celda cercana
(5, 0)
Celda cercana
(6, 1)
Celda cercana
(7, 0)
(6, 1)
Celda cercana
(5, 0)
Celda cercana
(6, 1)
Celda cercana
(7, 0)
(6, 1)
Nueva posicion final
(6, 1)
Celda cercana
(6, 2)
Celda cercana
(7, 1)
Celda cercana
(7, 3)
(7, 3)
Celda cercana
(6, 2)
Celda cercana
(7, 1)
Celda cercana
(7, 3)
(6, 2)
Nueva posicion final
(6, 2)
Going to coordinates
Nueva posicion final
(4, 3)
Counter: 
241
Celda cercana
(6, 0)
Celda cercana
(7, 1)
(7, 1)
Celda cercana
(6, 0)
Celda cercana
(7, 1)
(6, 0)
Celda cercana
(6, 0)
Celda cercana
(7, 1)
(7, 1)
Nueva posicion final
(7, 1)
Celda cercana
(0, 1)
Celda cercana
(1, 0)
(1, 0)
Nueva posicion final
(1, 0)
Going to coordinates
Nueva posicion final
(3, 3)
Celda cercana
(2, 4)
Celda cercana
(3, 5)
(3, 5)
Celda cercana
(2, 4)
Celda 

(1, 3)
Celda cercana
(4, 2)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
(5, 3)
Celda cercana
(4, 2)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
(5, 3)
Celda cercana
(4, 2)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
(5, 3)
Nueva posicion final
(5, 3)
Celda cercana
(3, 3)
Celda cercana
(4, 2)
(3, 3)
Celda cercana
(3, 3)
Celda cercana
(4, 2)
(4, 2)
Celda cercana
(3, 3)
Celda cercana
(4, 2)
(3, 3)
Nueva posicion final
(3, 3)
Celda cercana
(6, 5)
Celda cercana
(7, 4)
Celda cercana
(7, 6)
(7, 6)
Celda cercana
(6, 5)
Celda cercana
(7, 4)
Celda cercana
(7, 6)
(7, 6)
Nueva posicion final
(7, 6)
Counter: 
263
Celda cercana
(2, 3)
Celda cercana
(3, 2)
Celda cercana
(3, 4)
Celda cercana
(4, 3)
(3, 2)
Celda cercana
(2, 3)
Celda cercana
(3, 2)
Celda cercana
(3, 4)
Celda cercana
(4, 3)
(2, 3)
Celda cercana
(2, 3)
Celda cercana
(3, 2)
Celda cercana
(3, 4)
Celda cercana
(4, 3)
(3, 2)
Nueva posicion final
(3, 2)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
Celda cercana
(2, 6)
Celda cercana
(3, 5)
(3, 5)
Celda 

(3, 2)
Celda cercana
(3, 4)
Celda cercana
(4, 3)
(4, 3)
Celda cercana
(2, 3)
Celda cercana
(3, 2)
Celda cercana
(3, 4)
Celda cercana
(4, 3)
(3, 2)
Nueva posicion final
(3, 2)
Celda cercana
(4, 0)
Celda cercana
(4, 2)
(4, 0)
Celda cercana
(4, 0)
Celda cercana
(4, 2)
(4, 0)
Celda cercana
(4, 0)
Celda cercana
(4, 2)
(4, 2)
Nueva posicion final
(4, 2)
Celda cercana
(1, 3)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
(1, 5)
Celda cercana
(1, 3)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
(1, 3)
Celda cercana
(1, 3)
Celda cercana
(1, 5)
Celda cercana
(2, 4)
(1, 3)
Nueva posicion final
(1, 3)
Counter: 
279
Nueva posicion final
(3, 2)
Celda cercana
(2, 5)
Celda cercana
(3, 4)
Celda cercana
(3, 6)
Celda cercana
(4, 5)
(3, 4)
Celda cercana
(2, 5)
Celda cercana
(3, 4)
Celda cercana
(3, 6)
Celda cercana
(4, 5)
(4, 5)
Celda cercana
(2, 5)
Celda cercana
(3, 4)
Celda cercana
(3, 6)
Celda cercana
(4, 5)
(2, 5)
Nueva posicion final
(2, 5)
Celda cercana
(3, 7)
Celda cercana
(4, 6)
Celda cercana
(5, 7)
(3, 7)

Celda cercana
(3, 3)
(2, 4)
Celda cercana
(1, 3)
Celda cercana
(2, 2)
Celda cercana
(2, 4)
Celda cercana
(3, 3)
(1, 3)
Nueva posicion final
(1, 3)
Celda cercana
(0, 6)
Celda cercana
(1, 5)
(0, 6)
Celda cercana
(0, 6)
Celda cercana
(1, 5)
(0, 6)
Nueva posicion final
(0, 6)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
Celda cercana
(6, 4)
Celda cercana
(7, 3)
(7, 3)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
Celda cercana
(6, 4)
Celda cercana
(7, 3)
(7, 3)
Celda cercana
(5, 3)
Celda cercana
(6, 2)
Celda cercana
(6, 4)
Celda cercana
(7, 3)
(7, 3)
Nueva posicion final
(7, 3)
Counter: 
299
Celda cercana
(6, 4)
Celda cercana
(6, 6)
Celda cercana
(7, 5)
(7, 5)
Celda cercana
(6, 4)
Celda cercana
(6, 6)
Celda cercana
(7, 5)
(6, 4)
Celda cercana
(6, 4)
Celda cercana
(6, 6)
Celda cercana
(7, 5)
(7, 5)
Nueva posicion final
(7, 5)
Going to coordinates
Nueva posicion final
(5, 7)
Celda cercana
(0, 5)
Celda cercana
(0, 7)
Celda cercana
(1, 6)
(0, 5)
Celda cercana
(0, 5)
Celda cercana
(0, 7)
Celda cercana

(2, 5)
Celda cercana
(3, 4)
(1, 4)
Celda cercana
(1, 4)
Celda cercana
(2, 3)
Celda cercana
(2, 5)
Celda cercana
(3, 4)
(3, 4)
Celda cercana
(1, 4)
Celda cercana
(2, 3)
Celda cercana
(2, 5)
Celda cercana
(3, 4)
(1, 4)
Nueva posicion final
(1, 4)
Celda cercana
(5, 3)
Celda cercana
(6, 4)
(5, 3)
Celda cercana
(5, 3)
Celda cercana
(6, 4)
(5, 3)
Celda cercana
(5, 3)
Celda cercana
(6, 4)
(5, 3)
Nueva posicion final
(5, 3)
Celda cercana
(2, 1)
Celda cercana
(3, 0)
Celda cercana
(3, 2)
Celda cercana
(4, 1)
(2, 1)
Celda cercana
(2, 1)
Celda cercana
(3, 0)
Celda cercana
(3, 2)
Celda cercana
(4, 1)
(2, 1)
Celda cercana
(2, 1)
Celda cercana
(3, 0)
Celda cercana
(3, 2)
Celda cercana
(4, 1)
(4, 1)
Nueva posicion final
(4, 1)
Celda cercana
(6, 1)
Celda cercana
(7, 0)
Celda cercana
(7, 2)
(7, 2)
Celda cercana
(6, 1)
Celda cercana
(7, 0)
Celda cercana
(7, 2)
(6, 1)
Nueva posicion final
(6, 1)
Celda cercana
(4, 6)
Celda cercana
(5, 7)
(4, 6)
Celda cercana
(4, 6)
Celda cercana
(5, 7)
(5, 7)
Nueva posicio

## Visualización

In [5]:
todas_celdas = modelo.colectordatos.get_model_vars_dataframe()

In [6]:
print("Test")
print(todas_celdas.iloc[0][0])

Test
[[1. 1. 0. 6. 1. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1.]
 [6. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 1. 1. 6. 1. 0. 0.]
 [0. 6. 1. 0. 0. 0. 6. 0.]
 [1. 1. 0. 0. 0. 1. 0. 1.]]


In [7]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
something = todas_celdas.iloc[0][0]
patch = plt.imshow(todas_celdas.iloc[0][0], cmap='Purples')

def animate(i):
    patch.set_data(todas_celdas.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_celdas))

In [8]:
anim

In [9]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias:', tiempo_ejecucion)
print('Número de movimientos realizados por todos los agentes:', movimientos.tail()['Movimientos'].sum())

Tiempo necesario hasta que todas las celdas estén limpias: 0:00:03.797848
Número de movimientos realizados por todos los agentes: 1497.0
